In [ ]:
#libs itialization
import numpy as np
import matplotlib.pyplot as plt
from tkinter import *
from prettytable import PrettyTable

In [ ]:
Hu_code = {}
Sl_code = {}
Hu_name = []
Hu_value = []
th = []
td = []
fh = []
fd = []
text = r'123.txt'

In [ ]:
#Algorithm which searches the frequency of each english letter in the file
def findTheCharFrequency(text):
    freq = dict()
    with open(text,'r') as f:
        for line in f.readlines():
            line = line.lower()
            for i in line:
                if i.isalpha():
                    if i in freq:
                        freq[i] += 1
                    else:
                        freq.update({i:1})
    return freq
# sorted(l_frequency.items(), key=lambda item: item[1])

In [ ]:
#Printing first algorithm result as a table of letter frequency
l_frequency = findTheCharFrequency(text)
for key, value in sorted(l_frequency.items()):
    th.append(key)
    td.append(value)
columns = len(th)
f_table = PrettyTable(th)
while td:
    f_table.add_row(td[:columns])
    td = th[columns:]
print(f_table)

In [ ]:
# Создать класс узла
class Node(object):
    def __init__(self, name=None, value=None):
        self.name = name
        self.value = value
        self.lchild = None
        self.rchild = None

In [ ]:
# Huffman tree creation
class HuffmanTree(object):
    # Based on node we built the tree in reverse algorithm
    def __init__(self, char_Weights):
        self.Leaf = [Node(k,v) for k, v in char_Weights.items()]
        while len(self.Leaf) != 1:
            self.Leaf.sort(key=lambda node:node.value, reverse=True)
            n = Node(value=(self.Leaf[-1].value + self.Leaf[-2].value))
            n.lchild = self.Leaf.pop(-1)
            n.rchild = self.Leaf.pop(-1)
            self.Leaf.append(n)
        self.root = self.Leaf[0]
        self.Buffer = list(range(10))

    def Hu_generate(self, tree, length):
        node = tree
        if (not node):
            return
        elif node.name:
            n = node.name
            b = str()
            for i in range(length):
                b += str(self.Buffer[i])
            Hu_code[n] = b
            Hu_name.append(n)
            Hu_value.append(b)
            return
        self.Buffer[length] = 0
        self.Hu_generate(node.lchild, length + 1)
        self.Buffer[length] = 1
        self.Hu_generate(node.rchild, length + 1)

    #Output part
    def get_code(self):
        self.Hu_generate(self.root, 0)
    

#Printing full algorithm result
tree = HuffmanTree(l_frequency)
tree.get_code()
#the reason of duplication of function is that this is another sorting algorithm 
for key, value in sorted(l_frequency.items(), key=lambda item: item[1]): #sorted(l_frequency.items()): - previous one
    fh.append(key)
    fd.append(value)

for i in range (0, len(fh)):
    for j in range (0, len(fh)):
        if Hu_name[j] == fh[i]:
            fd[i] = [fd[i], Hu_value[j]]
columns = len(fh)
Hu_table = PrettyTable()
for i in range(0, len(fh)):
    Hu_table.add_column(fh[i], fd[i])
print(Hu_table)


# print(fd)
# print(l_frequency)
# print(Hu_code)



In [ ]:
zer0 = '00000'
for i in range (0, len(th)):
    Sl_code[th[i]] = zer0[:-(len(bin(i+1))-2)] + bin(i+1)[2:]
print(Sl_code)


In [ ]:
#here are functions for encryption with Huffman algorithm dictionary and with simular length dictionary
def HuEncryptWord(word):
    return ''.join([Hu_code[x] for x in word])
def SlEncryptWord(word):
    return ''.join([Sl_code[x] for x in word])

f = open('123.txt')
for word in f.read().split():
    print(HuEncryptWord(word))
    print(SlEncryptWord(word))
